In [38]:
import xml.etree.ElementTree as ET
import re
import pickle
import os

os.environ['PYWIKIBOT2_DIR'] = '../private/'
import pywikibot
import json
import logging


from collections import namedtuple
WikiGame = namedtuple('WikiGame', ('name', 'gapl_id', 'drm', 'steam_id', 'win', 'osx', 'lin'))
SteamID = namedtuple('SteamID', ('type', 'id'), defaults=(None, None))

def parse_gapl_data():
    
    gapl_data = []
    
    tree = ET.parse('../data_gapl/feed.xml')
    products = tree.getroot()
    
    for product in products:
        # Steam ID can exist regardless of delivery type
        gapl_steam_id = product.find('steam_id').text
        
        try:
            steam_id = SteamID(*gapl_steam_id.split('/'))
        except (ValueError, AttributeError):
            # Ignore bare Steam IDs as we don't know their type
            steam_id = SteamID()
        
        if steam_id.type == 'app':
            pass
        elif steam_id.type == 'sub':
            pass
        elif steam_id.type == 'bundle':
            pass

        # DRM can be (in order of occurences):
        # steam, uplay, origin, download, rockstarsocial, zenimax,
        # bethesdanet, giants, external, battlenet, arenanet, gog
        gapl_drm = product.find('delivery_type').text

        if gapl_drm == 'steam':
            drm = 'Steam'
        elif gapl_drm == 'uplay':
            drm = 'Uplay'
        elif gapl_drm == 'origin':
            drm = 'Origin'
        elif gapl_drm == 'bethesdanet':
            drm = 'Bethesda.net'
        elif gapl_drm == 'battlenet':
            drm = 'Battle.net'
        elif gapl_drm == 'gog':
            drm = 'GOG.com'
        else:
            # download, rockstarsocial, zenimax, giants, external, arenanet, ++
            drm = 'Unknown'
        
        platforms = product.find('platforms')
        win = platforms.find('pc').text == 'true'
        osx = platforms.find('mac').text == 'true'
        lin = platforms.find('linux').text == 'true'
        
        gapl_data.append(WikiGame(
            product.find('name').text,
            product.find('uid').text,
            drm,
            steam_id,
            win,
            osx,
            lin            
        ))
    return gapl_data

        
data = parse_gapl_data()

In [40]:
data[:5]

[WikiGame(name='Styx: Master of Shadows', gapl_id='2681-1', drm='Steam', steam_id=SteamID(type='app', id='242640'), win=True, osx=False, lin=False),
 WikiGame(name='Total War: MEDIEVAL II – Definitive Edition', gapl_id='1427-1', drm='Steam', steam_id=SteamID(type='app', id='4700'), win=True, osx=False, lin=False),
 WikiGame(name='Borderlands: The Pre-Sequel (Mac)', gapl_id='2680-1', drm='Steam', steam_id=SteamID(type='app', id='261640'), win=False, osx=True, lin=True),
 WikiGame(name='LEGO MARVEL: Super Heroes', gapl_id='1140-1', drm='Steam', steam_id=SteamID(type='app', id='249130'), win=True, osx=False, lin=False),
 WikiGame(name='Scribblenauts Unmasked: A DC Comics Adventure', gapl_id='1201-1', drm='Steam', steam_id=SteamID(type='app', id='249870'), win=True, osx=False, lin=False)]

In [42]:
gapl_data_s = sorted(data, key=lambda tup: tup[0])

In [43]:
gapl_data_s[:5]

[WikiGame(name='.hack//G.U. Last Recode', gapl_id='3450-1', drm='Steam', steam_id=SteamID(type='app', id='525480'), win=True, osx=False, lin=False),
 WikiGame(name='10 Second Ninja X', gapl_id='3362-1', drm='Steam', steam_id=SteamID(type='app', id='435790'), win=True, osx=False, lin=False),
 WikiGame(name='11-11 Memories Retold', gapl_id='3853-1', drm='Steam', steam_id=SteamID(type='app', id='735580'), win=True, osx=False, lin=False),
 WikiGame(name='15 Days', gapl_id='1823-1', drm='Steam', steam_id=SteamID(type='app', id='342990'), win=True, osx=False, lin=False),
 WikiGame(name='1943 Deadly Desert', gapl_id='3874-1', drm='Steam', steam_id=SteamID(type='app', id='616750'), win=True, osx=False, lin=False)]